In [314]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import requests
import json
from pandas import json_normalize

In [315]:
#url = "http://datamall2.mytransport.sg/ltaodataservice/PV/ODBus?Date=202402"

#payload = {}
#headers = {
#  'AccountKey': 'UCIaKN27Qi+uAjcDE+Zrog==',
#  'Accept': 'application/json'}

#response = requests.request("GET", url, headers=headers, data=payload)

#print(response.text)

In [316]:
#od_bus_1 = pd.read_csv('assets\original_destination_bus_stops\origin_destination_bus_202402.csv')
#od_bus_2 = pd.read_csv('assets\original_destination_bus_stops\origin_destination_bus_202403.csv')
#od_bus_3 = pd.read_csv('assets\original_destination_bus_stops\origin_destination_bus_202404.csv')
#od_bus_1.head()

# Passenger Volume by Bus Stops

In [317]:
import requests

url = "http://datamall2.mytransport.sg/ltaodataservice/PV/Bus?Date=202404"

payload = {}
headers = {
  'AccountKey': 'UCIaKN27Qi+uAjcDE+Zrog==',
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"odata.metadata":"http://datamall2.mytransport.sg/ltaodataservice/$metadata#FarecardBatch","value":[{"Link":"https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202404/transport_node_bus_202404.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEE8aDmFwLXNvdXRoZWFzdC0xIkYwRAIgSxFx6gc0P%2FVbXIXBmIMa6y7ofNySwp%2FlNAc4MU0KCTsCIDj5BM10Y38D%2FObb7913SFJns3YRoXh0T7OpfHYAcTAGKssFCOj%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMMzQwNjQ1MzgxMzA0IgxyX5%2FDtCZF0Je54t0qnwXmem2kyGOrSFQMsovQN4T7eC5RulFzXeskZwsAkt63JuzkWWPuiCRGXeMbTJhCVEE%2Fs3TF%2FQR7vVBS1tcspsMiQskXtu9dF%2FJDnuDk6roy%2Bi1DX8b3ABCebcyw7rM9XuOkaVqQAoyTCiw3vtgGhjtJNQv8IYY5IirVG8SRgSmPIyxOzvHqwAtZTiXymGNggHCHlQEvjeYmTozEn5oi6FSyivVHicorX6uow7aondaUKP0CV0oZLdq1214EG90UP0YOmicZk%2FCcn%2BwbGQZCjp7h6RPxhi97X6%2F2ATTPgG4aKVR7Bfzw21pKXS3YXhBdTO7ixgljQ64RjuvkuHoKHVfcO%2FIiwd1cZacIpZGJtzDClrU5Pq08QXz0gC%2Fp9eotgRdFH2eh3VGgrhBOtUsyy99yh1RON%2FZ0EeDJUhYaJJx6dxXgnJAGvNAqWvlWCYkXMiRQYOVlkvQ1QrrqZ9EXYhu45VCJeBJZJ84Sr2ERmIcSnI3HqM1wjyd6Yg0YqW5nBE8jzY57bipmIY1Zml7%2F%2

In [318]:
bus_1 = pd.read_csv('assets/bus_stops/transport_node_bus_202402.csv')
bus_2 = pd.read_csv('assets/bus_stops/transport_node_bus_202403.csv')
bus_3 = pd.read_csv('assets/bus_stops/transport_node_bus_202404.csv')
bus_3

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2024-04,WEEKENDS/HOLIDAY,17.0,BUS,45379,203,167
1,2024-04,WEEKDAY,17.0,BUS,45379,498,489
2,2024-04,WEEKDAY,13.0,BUS,80051,2711,2671
3,2024-04,WEEKENDS/HOLIDAY,13.0,BUS,80051,1408,1385
4,2024-04,WEEKENDS/HOLIDAY,13.0,BUS,5319,14,104
...,...,...,...,...,...,...,...
198774,2024-04,WEEKDAY,7.0,BUS,51011,2015,2092
198775,2024-04,WEEKENDS/HOLIDAY,18.0,BUS,70149,446,285
198776,2024-04,WEEKDAY,18.0,BUS,70149,2502,1732
198777,2024-04,WEEKDAY,6.0,BUS,13139,203,328


In [319]:
bus_3.dtypes

YEAR_MONTH               object
DAY_TYPE                 object
TIME_PER_HOUR           float64
PT_TYPE                  object
PT_CODE                   int64
TOTAL_TAP_IN_VOLUME       int64
TOTAL_TAP_OUT_VOLUME      int64
dtype: object

In [320]:
import pandas as pd

def preprocess_data(data):
    # Remove null values
    data = data.dropna()

    # Remove the "HOLIDAY" remark to just include weekend
    data['DAY_TYPE'] = data['DAY_TYPE'].str.replace('S/HOLIDAY', '')

    # Update the DAY_TYPE column to lowercase
    data.loc[:, 'DAY_TYPE'] = data['DAY_TYPE'].str.replace('S/HOLIDAY', '').str.lower()

    # Update TIME_PER_HOUR to remove the decimal
    data.loc[:, 'TIME_PER_HOUR'] = data['TIME_PER_HOUR'].astype(int)
    data['TIME_PER_HOUR'] = data['TIME_PER_HOUR'].apply(lambda x: '{:02d}'.format(x))

    # Remove PT_TYPE column
    data = data.drop(columns=['PT_TYPE'])

    # Create a new column to sum the passengers tap in and out
    data['crowd'] = data['TOTAL_TAP_IN_VOLUME']

    # Combine the DAY_TYPE and TIME_PER_HOUR
    data['day_type_time'] = data['DAY_TYPE'] + "_" + data['TIME_PER_HOUR'].astype(str)

    # Drop the unnecessary columns
    data = data.drop(columns=['TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME', 'YEAR_MONTH', 'DAY_TYPE', 'TIME_PER_HOUR'])

    return data

# Example usage:
# Assuming you have a DataFrame named bus_3
# preprocessed_bus_3 = preprocess_bus_data(bus_3)

In [321]:
bus_april = preprocess_data(bus_3)
bus_april

C:\Users\IINDR\AppData\Local\Temp\ipykernel_11928\2012527744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DAY_TYPE'] = data['DAY_TYPE'].str.replace('S/HOLIDAY', '')
C:\Users\IINDR\AppData\Local\Temp\ipykernel_11928\2012527744.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'DAY_TYPE'] = data['DAY_TYPE'].str.replace('S/HOLIDAY', '').str.lower()
C:\Users\IINDR\AppData\Local\Temp\ipykernel_11928\2012527744.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,PT_CODE,crowd,day_type_time
0,45379,203,weekend_17
1,45379,498,weekday_17
2,80051,2711,weekday_13
3,80051,1408,weekend_13
4,5319,14,weekend_13
...,...,...,...
198774,51011,2015,weekday_07
198775,70149,446,weekend_18
198776,70149,2502,weekday_18
198777,13139,203,weekday_06


### Bus Stop Coordinate

In [322]:
import json

# Path to your JSON file
file_path = 'assets/bus_stops/bus_stop_coordinate.json'

# Read JSON data from file
with open(file_path, 'r') as file:
    data = json.load(file)

# Convert to DataFrame
bus_stop_coordinate = pd.DataFrame(data['value'])

# Display the DataFrame
bus_stop_coordinate

,Latitude,Description,Longitude,RoadName,BusStopCode
0,1.383764,BT PANJANG TEMP BUS PK,103.758301,Woodlands Rd,00481
1,1.296848,Hotel Grand Pacific,103.852539,Victoria St,01012
2,1.297710,St. Joseph's Ch,103.853226,Victoria St,01013
3,1.296990,Bras Basah Cplx,103.853020,Victoria St,01019
4,1.296673,Cosmic Insurance Bldg,103.854416,Nth Bridge Rd,01029
...,...,...,...,...,...
4971,1.400663,Blk 661,103.750854,Choa Chu Kang Cres,N4541
4972,1.402724,Blk 666,103.750015,Choa Chu Kang Cres,N4559
4973,1.403138,Blk 669,103.748489,Choa Chu Kang Cres,N4569
4974,1.401517,Blk 675,103.745834,Choa Chu Kang Cres,N4579


In [323]:
# Remove the first character from 'BusStopCode' column if it's a letter
bus_stop_coordinate['BusStopCode'] = bus_stop_coordinate['BusStopCode'].str.replace(r'^\D+', '', regex=True)

# Change the data type of the 'BusStopCode' column from object to int
bus_stop_coordinate['BusStopCode'] = bus_stop_coordinate['BusStopCode'].astype(int)

In [324]:
bus_stop_coordinate.dtypes

Latitude       float64
Description     object
Longitude      float64
RoadName        object
BusStopCode      int32
dtype: object

In [325]:
bus_stop_coordinate = bus_stop_coordinate.drop(columns = ['Description', 'RoadName'])
bus_stop_coordinate

,Latitude,Longitude,BusStopCode
0,1.383764,103.758301,481
1,1.296848,103.852539,1012
2,1.297710,103.853226,1013
3,1.296990,103.853020,1019
4,1.296673,103.854416,1029
...,...,...,...
4971,1.400663,103.750854,4541
4972,1.402724,103.750015,4559
4973,1.403138,103.748489,4569
4974,1.401517,103.745834,4579


#### Merge the passenger volume data with bus stop coordinate

In [326]:
bus_april.head()

,PT_CODE,crowd,day_type_time
0,45379,203,weekend_17
1,45379,498,weekday_17
2,80051,2711,weekday_13
3,80051,1408,weekend_13
4,5319,14,weekend_13


In [327]:
bus_stop_coordinate.head()

,Latitude,Longitude,BusStopCode
0,1.383764,103.758301,481
1,1.296848,103.852539,1012
2,1.297710,103.853226,1013
3,1.296990,103.853020,1019
4,1.296673,103.854416,1029


In [328]:
bus_april_df = pd.merge(bus_april, bus_stop_coordinate, left_on='PT_CODE', right_on='BusStopCode', how='inner')
bus_april_df

,PT_CODE,crowd,day_type_time,Latitude,Longitude,BusStopCode
0,45379,203,weekend_17,1.394588,103.751869,45379
1,45379,498,weekday_17,1.394588,103.751869,45379
2,45379,4,weekend_00,1.394588,103.751869,45379
3,45379,18,weekday_00,1.394588,103.751869,45379
4,45379,80,weekend_20,1.394588,103.751869,45379
...,...,...,...,...,...,...
190935,43051,295,weekend_21,1.359102,103.767052,43051
190936,43051,335,weekend_19,1.359102,103.767052,43051
190937,43051,791,weekday_19,1.359102,103.767052,43051
190938,43051,59,weekday_00,1.359102,103.767052,43051


In [329]:
bus_april_df['PT_CODE'].nunique()

4902

In [330]:
bus_april_df = bus_april_df.drop(columns = ['PT_CODE', 'BusStopCode'])
bus_april_df

,crowd,day_type_time,Latitude,Longitude
0,203,weekend_17,1.394588,103.751869
1,498,weekday_17,1.394588,103.751869
2,4,weekend_00,1.394588,103.751869
3,18,weekday_00,1.394588,103.751869
4,80,weekend_20,1.394588,103.751869
...,...,...,...,...
190935,295,weekend_21,1.359102,103.767052
190936,335,weekend_19,1.359102,103.767052
190937,791,weekday_19,1.359102,103.767052
190938,59,weekday_00,1.359102,103.767052


In [331]:
# One-hot encode 'day_type_time' column
bus_april_df = pd.get_dummies(bus_april_df, columns=['day_type_time'])

bus_april_df

,crowd,Latitude,Longitude,day_type_time_weekday_00,day_type_time_weekday_01,day_type_time_weekday_02,day_type_time_weekday_03,day_type_time_weekday_04,day_type_time_weekday_05,day_type_time_weekday_06,day_type_time_weekday_07,day_type_time_weekday_08,day_type_time_weekday_09,day_type_time_weekday_10,day_type_time_weekday_11,day_type_time_weekday_12,day_type_time_weekday_13,day_type_time_weekday_14,day_type_time_weekday_15,day_type_time_weekday_16,day_type_time_weekday_17,day_type_time_weekday_18,day_type_time_weekday_19,day_type_time_weekday_20,day_type_time_weekday_21,day_type_time_weekday_22,day_type_time_weekday_23,day_type_time_weekend_00,day_type_time_weekend_01,day_type_time_weekend_02,day_type_time_weekend_04,day_type_time_weekend_05,day_type_time_weekend_06,day_type_time_weekend_07,day_type_time_weekend_08,day_type_time_weekend_09,day_type_time_weekend_10,day_type_time_weekend_11,day_type_time_weekend_12,day_type_time_weekend_13,day_type_time_weekend_14,day_type_time_weekend_15,day_type_time_weekend_16,day_type_time_weekend_17,day_type_time_weekend_18,day_type_time_weekend_19,day_type_time_weekend_20,day_type_time_weekend_21,day_type_time_weekend_22,day_type_time_weekend_23
0,203,1.394588,103.751869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,498,1.394588,103.751869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,1.394588,103.751869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,18,1.394588,103.751869,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,80,1.394588,103.751869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190935,295,1.359102,103.767052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
190936,335,1.359102,103.767052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
190937,791,1.359102,103.767052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
190938,59,1.359102,103.767052,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [332]:
for i in bus_april_df.columns[3:]:
    bus_april_df[i] = bus_april_df[i] * bus_april_df['crowd']

bus_april_df = bus_april_df.drop(columns='crowd')

bus_april_df.columns = bus_april_df.columns.str.replace('day_type_time_', '')

bus_april_df

,Latitude,Longitude,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,weekday_05,weekday_06,weekday_07,weekday_08,weekday_09,weekday_10,weekday_11,weekday_12,weekday_13,weekday_14,weekday_15,weekday_16,weekday_17,weekday_18,weekday_19,weekday_20,weekday_21,weekday_22,weekday_23,weekend_00,weekend_01,weekend_02,weekend_04,weekend_05,weekend_06,weekend_07,weekend_08,weekend_09,weekend_10,weekend_11,weekend_12,weekend_13,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,1.394588,103.751869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,203,0,0,0,0,0,0
1,1.394588,103.751869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.394588,103.751869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.394588,103.751869,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.394588,103.751869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190935,1.359102,103.767052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,295,0,0
190936,1.359102,103.767052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,335,0,0,0,0
190937,1.359102,103.767052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,791,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
190938,1.359102,103.767052,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [333]:
#url = "http://datamall2.mytransport.sg/ltaodataservice/PV/ODTrain?Date=202402"

#payload = {}
#headers = {
#  'AccountKey': '9NKPmJIoQaWOFcIhkALlxg==',
#  'Accept': 'application/json'}

#response = requests.request("GET", url, headers=headers, data=payload)

#print(response.text)

In [334]:
#od_bus_1 = pd.read_csv('assets\original_destination_train_stations\origin_destination_train_202402.csv')
#od_bus_2 = pd.read_csv('assets\original_destination_train_stations\origin_destination_train_202403.csv')
#od_bus_3 = pd.read_csv('assets\original_destination_train_stations\origin_destination_train_202404.csv')
#od_bus_1.head()

## Passenger Volume by Train Stations

In [335]:
import requests

url = "http://datamall2.mytransport.sg/ltaodataservice/PV/Train?Date=202402"

payload = {}
headers = {
  'AccountKey': '3ymKgbTFSaWeeGRwjZyHbw==',
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

In [336]:
train_1 = pd.read_csv('assets/train_stations/transport_node_train_202402.csv')
train_2 = pd.read_csv('assets/train_stations/transport_node_train_202403.csv')
train_3 = pd.read_csv('assets/train_stations/transport_node_train_202404.csv')
train_3.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2024-04,WEEKDAY,22,TRAIN,NS28,901,457
1,2024-04,WEEKENDS/HOLIDAY,22,TRAIN,NS28,287,192
2,2024-04,WEEKENDS/HOLIDAY,0,TRAIN,DT10/TE11,37,204
3,2024-04,WEEKDAY,0,TRAIN,DT10/TE11,74,343
4,2024-04,WEEKENDS/HOLIDAY,10,TRAIN,EW16/NE3/TE17,8751,10141


In [337]:
train_april = preprocess_data(train_3)
train_april.head(10)

C:\Users\IINDR\AppData\Local\Temp\ipykernel_11928\2012527744.py:14: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[:, 'TIME_PER_HOUR'] = data['TIME_PER_HOUR'].astype(int)


,PT_CODE,crowd,day_type_time
0,NS28,901,weekday_22
1,NS28,287,weekend_22
2,DT10/TE11,37,weekend_00
3,DT10/TE11,74,weekday_00
4,EW16/NE3/TE17,8751,weekend_10
5,EW16/NE3/TE17,26579,weekday_10
6,CC14,3725,weekend_09
7,CC14,11849,weekday_09
8,EW27,31232,weekday_14
9,EW27,21848,weekend_14


### MRT Station Coordinate

In [338]:
mrt_station_coordinate = pd.read_csv('assets/train_stations/mrt_station_coordinate.csv')
mrt_station_coordinate

,Unnamed: 0,OBJECTID,STN_NAME,STN_NO,geometry,Latitude,Longitude
0,0,1,EUNOS MRT STATION,EW7,POINT (103.9032524667383 1.319778951553637),1.319779,103.903252
1,1,2,CHINESE GARDEN MRT STATION,EW25,POINT (103.7325967380734 1.342352820874744),1.342353,103.732597
2,2,3,KHATIB MRT STATION,NS14,POINT (103.8329799077383 1.417383370153547),1.417383,103.832980
3,3,4,KRANJI MRT STATION,NS7,POINT (103.7621654109002 1.425177698770448),1.425178,103.762165
4,4,5,REDHILL MRT STATION,EW18,POINT (103.816816670149 1.289562726402453),1.289563,103.816817
...,...,...,...,...,...,...,...
166,166,198,SPRINGLEAF MRT STATION,TE4,POINT (103.8180818498627 1.398160861025955),1.398161,103.818082
167,167,197,LENTOR MRT STATION,TE5,POINT (103.8364694869142 1.385061946926286),1.385062,103.836469
168,168,196,MAYFLOWER MRT STATION,TE6,POINT (103.8368239320149 1.372086638674201),1.372087,103.836824
169,169,195,BRIGHT HILL MRT STATION,TE7,POINT (103.8329359578363 1.363308098095808),1.363308,103.832936


In [339]:
mrt_station_coordinate = mrt_station_coordinate.drop(columns = ['Unnamed: 0', 'OBJECTID', 'STN_NAME'])
mrt_station_coordinate

,STN_NO,geometry,Latitude,Longitude
0,EW7,POINT (103.9032524667383 1.319778951553637),1.319779,103.903252
1,EW25,POINT (103.7325967380734 1.342352820874744),1.342353,103.732597
2,NS14,POINT (103.8329799077383 1.417383370153547),1.417383,103.832980
3,NS7,POINT (103.7621654109002 1.425177698770448),1.425178,103.762165
4,EW18,POINT (103.816816670149 1.289562726402453),1.289563,103.816817
...,...,...,...,...
166,TE4,POINT (103.8180818498627 1.398160861025955),1.398161,103.818082
167,TE5,POINT (103.8364694869142 1.385061946926286),1.385062,103.836469
168,TE6,POINT (103.8368239320149 1.372086638674201),1.372087,103.836824
169,TE7,POINT (103.8329359578363 1.363308098095808),1.363308,103.832936


#### Merge the passenger volume data with MRT station coordinate

In [340]:
mrt_set = set(mrt_station_coordinate['STN_NO'])
train_set = set(train_april['PT_CODE'])

in_mrt_not_in_train = mrt_set - train_set
in_mrt_not_in_train

{'BP1',
 'BP6',
 'CC18',
 'DT1',
 'DT16/CE1',
 'DT4',
 'EW13/NS25',
 'EW16',
 'NE16',
 'NE17',
 'NS22/TE14',
 'NS27/TE20/CE2',
 'NS4',
 'PTC',
 'PW2',
 'STC'}

In [341]:
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('BP1', 'NS4/BP1')
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('BP6', 'BP6/DT1')
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('DT1', 'BP6/DT1')
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('DT16/CE1', 'CE1/DT16')

mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('EW13/NS25', 'NS25/EW13')
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('EW16', 'EW16/NE3/TE17')

mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('NE16', 'NE16/STC')
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('NE17', 'NE17/PTC')

mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('NE16', 'NE16/STC')
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('NE17', 'NE17/PTC')

mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('NS22/TE14', 'TE14/NS22')
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('NS27/TE20/CE2', 'NS27/CE2/TE20')

mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('NS4', 'NS4/BP1')
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('PTC', 'NE17/PTC')
mrt_station_coordinate['STN_NO'] = mrt_station_coordinate['STN_NO'].replace('STC', 'NE16/STC')

mrt_station_coordinate

,STN_NO,geometry,Latitude,Longitude
0,EW7,POINT (103.9032524667383 1.319778951553637),1.319779,103.903252
1,EW25,POINT (103.7325967380734 1.342352820874744),1.342353,103.732597
2,NS14,POINT (103.8329799077383 1.417383370153547),1.417383,103.832980
3,NS7,POINT (103.7621654109002 1.425177698770448),1.425178,103.762165
4,EW18,POINT (103.816816670149 1.289562726402453),1.289563,103.816817
...,...,...,...,...
166,TE4,POINT (103.8180818498627 1.398160861025955),1.398161,103.818082
167,TE5,POINT (103.8364694869142 1.385061946926286),1.385062,103.836469
168,TE6,POINT (103.8368239320149 1.372086638674201),1.372087,103.836824
169,TE7,POINT (103.8329359578363 1.363308098095808),1.363308,103.832936


In [342]:
train_april_df = pd.merge(train_april, mrt_station_coordinate, left_on='PT_CODE', right_on='STN_NO', how='inner')
train_april_df.head(10)

,PT_CODE,crowd,day_type_time,STN_NO,geometry,Latitude,Longitude
0,NS28,901,weekday_22,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
1,NS28,287,weekend_22,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
2,NS28,342,weekend_08,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
3,NS28,902,weekday_08,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
4,NS28,282,weekend_07,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
5,NS28,566,weekday_07,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
6,NS28,3307,weekday_18,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
7,NS28,4401,weekend_18,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
8,NS28,1345,weekday_12,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
9,NS28,3796,weekend_12,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286


In [343]:
train_april_df.head()

,PT_CODE,crowd,day_type_time,STN_NO,geometry,Latitude,Longitude
0,NS28,901,weekday_22,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
1,NS28,287,weekend_22,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
2,NS28,342,weekend_08,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
3,NS28,902,weekday_08,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
4,NS28,282,weekend_07,NS28,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286


In [344]:
train_april_df = train_april_df.drop(columns = ['PT_CODE' , 'STN_NO'])
train_april_df.head()

,crowd,day_type_time,geometry,Latitude,Longitude
0,901,weekday_22,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
1,287,weekend_22,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
2,342,weekend_08,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
3,902,weekday_08,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286
4,282,weekend_07,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286


In [345]:
# One-hot encode 'day_type_time' column
pd.set_option('display.max_columns', None)
train_april_final = pd.get_dummies(train_april_df, columns=['day_type_time'])
train_april_final

,crowd,geometry,Latitude,Longitude,day_type_time_weekday_00,day_type_time_weekday_05,day_type_time_weekday_06,day_type_time_weekday_07,day_type_time_weekday_08,day_type_time_weekday_09,day_type_time_weekday_10,day_type_time_weekday_11,day_type_time_weekday_12,day_type_time_weekday_13,day_type_time_weekday_14,day_type_time_weekday_15,day_type_time_weekday_16,day_type_time_weekday_17,day_type_time_weekday_18,day_type_time_weekday_19,day_type_time_weekday_20,day_type_time_weekday_21,day_type_time_weekday_22,day_type_time_weekday_23,day_type_time_weekend_00,day_type_time_weekend_05,day_type_time_weekend_06,day_type_time_weekend_07,day_type_time_weekend_08,day_type_time_weekend_09,day_type_time_weekend_10,day_type_time_weekend_11,day_type_time_weekend_12,day_type_time_weekend_13,day_type_time_weekend_14,day_type_time_weekend_15,day_type_time_weekend_16,day_type_time_weekend_17,day_type_time_weekend_18,day_type_time_weekend_19,day_type_time_weekend_20,day_type_time_weekend_21,day_type_time_weekend_22,day_type_time_weekend_23
0,901,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,287,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,342,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,902,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,282,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6696,5713,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6697,3646,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6698,6732,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6699,4870,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [346]:
train_april_final.head()

,crowd,geometry,Latitude,Longitude,day_type_time_weekday_00,day_type_time_weekday_05,day_type_time_weekday_06,day_type_time_weekday_07,day_type_time_weekday_08,day_type_time_weekday_09,day_type_time_weekday_10,day_type_time_weekday_11,day_type_time_weekday_12,day_type_time_weekday_13,day_type_time_weekday_14,day_type_time_weekday_15,day_type_time_weekday_16,day_type_time_weekday_17,day_type_time_weekday_18,day_type_time_weekday_19,day_type_time_weekday_20,day_type_time_weekday_21,day_type_time_weekday_22,day_type_time_weekday_23,day_type_time_weekend_00,day_type_time_weekend_05,day_type_time_weekend_06,day_type_time_weekend_07,day_type_time_weekend_08,day_type_time_weekend_09,day_type_time_weekend_10,day_type_time_weekend_11,day_type_time_weekend_12,day_type_time_weekend_13,day_type_time_weekend_14,day_type_time_weekend_15,day_type_time_weekend_16,day_type_time_weekend_17,day_type_time_weekend_18,day_type_time_weekend_19,day_type_time_weekend_20,day_type_time_weekend_21,day_type_time_weekend_22,day_type_time_weekend_23
0,901,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,287,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,342,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,902,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,282,POINT (103.8628597696374 1.27133671132917),1.271337,103.86286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [347]:
pd.set_option('display.max_columns', None)
for i in train_april_final.columns[4:]:
    train_april_final[i] = train_april_final[i] * train_april_final['crowd']

train_april_final = train_april_final.drop(columns='crowd')

train_april_final.columns = train_april_final.columns.str.replace('day_type_time_', '')

train_april_final

,geometry,Latitude,Longitude,weekday_00,weekday_05,weekday_06,weekday_07,weekday_08,weekday_09,weekday_10,weekday_11,weekday_12,weekday_13,weekday_14,weekday_15,weekday_16,weekday_17,weekday_18,weekday_19,weekday_20,weekday_21,weekday_22,weekday_23,weekend_00,weekend_05,weekend_06,weekend_07,weekend_08,weekend_09,weekend_10,weekend_11,weekend_12,weekend_13,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,901,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,287,0
2,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,342,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,902,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,282,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6696,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,0,0,0,0,0,5713,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6697,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3646,0,0,0,0,0,0,0,0,0,0
6698,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,0,0,6732,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6699,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4870,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [350]:
train_april_final = train_april_final.drop(columns = 'geometry')
train_april_final.head()

,latitude,longitude,weekday_00,weekday_05,weekday_06,weekday_07,weekday_08,weekday_09,weekday_10,weekday_11,weekday_12,weekday_13,weekday_14,weekday_15,weekday_16,weekday_17,weekday_18,weekday_19,weekday_20,weekday_21,weekday_22,weekday_23,weekend_00,weekend_05,weekend_06,weekend_07,weekend_08,weekend_09,weekend_10,weekend_11,weekend_12,weekend_13,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,1.271337,103.86286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,901,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.271337,103.86286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,287,0
2,1.271337,103.86286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,342,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.271337,103.86286,0,0,0,0,902,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.271337,103.86286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,282,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [348]:
# Convert column names to lower case
bus_april_df.columns = bus_april_df.columns.str.lower()
train_april_final.columns = train_april_final.columns.str.lower()

In [349]:
train_april_final.to_csv('assets/processed_data/train_volume.csv')
bus_april_df.to_csv('assets/processed_data/bus_volume.csv')